In [1]:
import pandas as pd
from pandas import option_context

import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

import re
import string

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.decomposition import NMF

from nltk.corpus import stopwords
from nltk.tokenize import MWETokenizer

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 

## Intent analysis

#### Naive Bayes

In [306]:
df_intent_tr = pd.read_csv('training_base.csv')

In [307]:
def remove_space(text):
    text = text.strip()
    return text
    
df_intent_tr['tweet_compound'] = df_intent_tr.tweet_compound.map(remove_space)

In [429]:
df_intent_tr.dropna(inplace=True)
df_intent_tr.reset_index(drop=True,inplace=True)
df_intent_tr.shape

(2995, 10)

In [311]:
df_intent_tr.groupby('intent').count()

,Unnamed: 0,username,tweet,tweet_compound,topic,neg,neu,pos,compound
intent,,,,,,,,,
1,1140,1140,1140,1140,1140,1140,1140,1140,1140
2,1217,1217,1217,1217,1217,1217,1217,1217,1217
3,27,27,27,27,27,27,27,27,27
4,184,184,184,184,184,184,184,184,184
5,84,84,84,84,84,84,84,84,84
6,120,120,120,120,120,120,120,120,120
9,223,223,223,223,223,223,223,223,223


In [312]:
X_tr = df_intent_tr['tweet_compound']
X_tr.shape

(2995,)

In [313]:
y_tr = df_intent_tr[['intent']]

In [314]:
# creating bag of words model 
cv = CountVectorizer(max_features=1100) 
  
X = cv.fit_transform(X_tr).toarray() 
y = y_tr.iloc[:, 0].values 

# fitting naive bayes to the training set 
from sklearn.naive_bayes import MultinomialNB 

classifier = MultinomialNB();
classifier.fit(X, y) 

MultinomialNB()

In [315]:
df_intent_pred = pd.read_csv('test5.csv')
df_intent_pred['tweet_compound'] = df_intent_pred.tweet_compound.map(remove_space)
df_intent_pred.shape

(200, 9)

In [316]:
X_test = df_intent_pred['tweet_compound']
X_t2 = cv.fit_transform(X_test).toarray() 

In [317]:
y_pred = classifier.predict(X_t2) 

In [318]:
df_intent_pred['pred'] =  classifier.predict(X_t2)

In [319]:
df_intent_pred

,Unnamed: 0,username,tweet,tweet_compound,topic,neg,neu,pos,compound,pred
0,69028,elyseon_01,no reset methods seem to be working of th...,no reset methods seem to be working of the ...,4,0.087,0.913,0.000,-0.5574,5
1,54998,elithewalrus,he commented on this post too yesterday,he commented on this post too yesterday,4,0.000,1.000,0.000,0.0000,5
2,59191,brendastevenage,please follow me you may find it interesting...,please follow me you may find it interesting t...,2,0.238,0.701,0.061,-0.8779,5
3,94356,jamessential,seriously revolut has anyone mentioned how an...,seriously revolut has anyone mentioned how ant...,1,0.061,0.939,0.000,-0.1779,3
4,54609,motwitrer,hello i have lost access to my revolut accoun...,hello i have lost access to my revolut account...,1,0.032,0.911,0.057,0.3182,3
...,...,...,...,...,...,...,...,...,...,...
195,18444,sbliudze,hey just saying your travel insurance coverag...,hey just saying your travel insurance coverag...,1,0.047,0.904,0.049,0.0258,5
196,88395,vip_help1,i have deposited money am trying to verify my...,i have deposited money am trying to verify my ...,3,0.000,1.000,0.000,0.0000,3
197,3893,joshey_r,what is going on with your support other bank...,what is going on with your support other banks...,4,0.024,0.704,0.271,0.9246,5
198,70983,erwanthiboumery,please it is impossible de connecting my rev...,please it is impossible de connecting my revo...,1,0.000,0.759,0.241,0.7430,5


#### Round 2 Naive Bayes

In [430]:
df_intent_tr2 = pd.read_csv('training_base.csv')

In [431]:
df_intent_tr2['tweet_compound'] = df_intent_tr2.tweet_compound.map(remove_space)

In [432]:
df_intent_tr2.dropna(inplace=True)
df_intent_tr2.reset_index(drop=True,inplace=True)
df_intent_tr2.shape

(3189, 10)

In [433]:
df_intent_tr2.groupby('intent').count()

,Unnamed: 0,username,tweet,tweet_compound,topic,neg,neu,pos,compound
intent,,,,,,,,,
0,1211,1211,1211,1211,1211,1211,1211,1211,1211
1,30,30,30,30,30,30,30,30,30
2,1287,1287,1287,1287,1287,1287,1287,1287,1287
3,186,186,186,186,186,186,186,186,186
4,323,323,323,323,323,323,323,323,323
5,152,152,152,152,152,152,152,152,152


In [434]:
X_tr2 = df_intent_tr2['tweet_compound']
X_tr2.shape

(3189,)

In [435]:
y_tr2 = df_intent_tr2[['intent']]

In [436]:
# creating bag of words model 
cv2 = CountVectorizer(max_features=1100) 
  
X2 = cv2.fit_transform(X_tr2).toarray() 
y2 = y_tr2.iloc[:, 0].values 

# fitting naive bayes to the training set 

classifier2 = MultinomialNB();
classifier2.fit(X2, y2) 

MultinomialNB()

In [437]:
df_intent_pred2 = pd.read_csv('test_preds7_xgb.csv')
df_intent_pred2['tweet_compound'] = df_intent_pred2.tweet_compound.map(remove_space)
df_intent_pred2.shape

(200, 12)

In [438]:
X_test2 = df_intent_pred2['tweet_compound']
X_t2_2 = cv2.fit_transform(X_test2).toarray() 

In [439]:
y_pred2 = classifier2.predict(X_t2_2) 

In [440]:
df_intent_pred2['pred_nb'] =  classifier2.predict(X_t2_2)

In [441]:
accuracy2 = accuracy_score(df_intent_pred2['intent'], y_pred2)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 30.00%


In [452]:
df_intent_pred2.sample(10)

,Unnamed: 0,Unnamed: 0.1,username,tweet,tweet_compound,intent,topic,neg,neu,pos,compound,gbm_pred,pred_nb
164,164,38212,dovydas57453143,is there any support number that i could call ...,is there any support number that i could call ...,2,2,0.044,0.829,0.127,0.3182,2,1
46,46,95343,shay_uk,my chat is showing as offline can you help,my chat is showing as offline can you help,2,4,0.134,0.625,0.241,0.2960,2,1
141,141,31364,jaymcdonnell151,when do ye think this is gonna be fixed absolu...,when do ye think this is gonna be fixed absolu...,0,3,0.000,0.891,0.109,0.2960,2,5
16,16,86252,adutvincent,thanks everyone that is asking for my review a...,thanks everyone that is asking for my review a...,0,2,0.000,0.890,0.110,0.6369,1,1
132,132,84851,young_rxco,whats going on with app this morning i cant...,whats going on with app this morning i cant...,2,1,0.000,0.957,0.043,0.3182,3,1
139,139,95017,bojanmilicevic,this was written a year ago application for a...,this was written a year ago app for a europea...,0,1,0.044,0.892,0.064,0.2247,2,5
9,9,87278,tompickering,go for it support has been good so far althou...,go for it support has been good so far althou...,3,1,0.044,0.856,0.100,0.2382,2,1
130,130,17182,_noodles__,launch fully in canada,launch fully in canada,3,2,0.000,1.000,0.000,0.0000,2,0
87,87,71294,wennygl,im already following you,im already following you,2,4,0.000,1.000,0.000,0.0000,2,4
74,74,37468,cyiaphmccann,got it sorted through chat thanks,got it sorted through chat thanks,3,4,0.000,0.633,0.367,0.4404,2,5


## With XGBoost

In [170]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

In [487]:
df_intent_tr7 = pd.read_csv('training_base.csv')
df_intent_tr7.dropna(inplace=True)

In [488]:
df_intent_tr7.groupby('intent').count()

,Unnamed: 0,username,tweet,tweet_compound,topic,neg,neu,pos,compound
intent,,,,,,,,,
0,1211,1211,1211,1211,1211,1211,1211,1211,1211
1,30,30,30,30,30,30,30,30,30
2,1287,1287,1287,1287,1287,1287,1287,1287,1287
3,186,186,186,186,186,186,186,186,186
4,323,323,323,323,323,323,323,323,323
5,152,152,152,152,152,152,152,152,152


In [489]:
X_tr7 = df_intent_tr7['tweet_compound']
X_tr7.shape

(3189,)

In [490]:
y_tr7 = df_intent_tr7[['intent']]

In [491]:
df_test7 = pd.read_csv('test_preds7_xgb.csv')
df_test7['tweet_compound'] = df_test7.tweet_compound.map(remove_space)
df_test7.shape

(200, 12)

In [492]:
# creating bag of words model 
cv7 = CountVectorizer(stop_words = stop,
                    max_features=950) 
  
X7 = cv7.fit_transform(X_tr7).toarray() 
y7 = y_tr7.iloc[:, 0].values 

# fitting XGB to the training set 
gbm7 = xgb.XGBRegressor(max_depth=5,
                        learning_rate = 0.05,
                        objective = 'multi:softmax',
                        num_class = 6)

/Users/tawneykirkland/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['course'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [493]:
gbm7.fit(X7,y7)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=5,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_class=6, num_parallel_tree=1,
             objective='multi:softmax', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=None, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [494]:
df_test7_7 = df_test7['tweet_compound']
X_t7 = cv7.fit_transform(df_test7_7).toarray() 
y_pred7 = gbm7.predict(X_t7)
gbm_pred7 = [round(value) for value in y_pred7]

In [499]:
accuracy = accuracy_score(df_test7['intent'], gbm_pred7)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 32.50%


In [500]:
df_test7['gbm_pred'] =  gbm_pred7

In [501]:
df_test7.groupby('gbm_pred').count()

,Unnamed: 0,Unnamed: 0.1,username,tweet,tweet_compound,intent,topic,neg,neu,pos,compound
gbm_pred,,,,,,,,,,,
0,55,55,55,55,55,55,55,55,55,55,55
2,129,129,129,129,129,129,129,129,129,129,129
3,11,11,11,11,11,11,11,11,11,11,11
4,4,4,4,4,4,4,4,4,4,4,4
5,1,1,1,1,1,1,1,1,1,1,1


In [502]:
df_test7.head(10)

,Unnamed: 0,Unnamed: 0.1,username,tweet,tweet_compound,intent,topic,neg,neu,pos,compound,gbm_pred
0,0,91349,sheesbees,so do i need a credit card to use the revolut ...,so do i need a creditcard to use the debitcard...,2,2,0.000,1.000,0.000,0.0000,2
1,1,87363,ncsh,iz nize proves the breakout growth potentia...,iz nize proves the breakout growth potentia...,3,1,0.000,0.775,0.225,0.6597,2
2,2,72737,johnchambers81,hi can you dm me please regarding the app bein...,hi can you dm me please regarding the app bein...,2,1,0.065,0.868,0.067,0.0108,0
3,3,60257,jpoliv,hey revolut how do you explain this why do you...,hey revolut how do you explain this why do you...,2,2,0.101,0.866,0.032,-0.5859,0
4,4,12232,fintechris,get your finger out now,get your finger out now,4,1,0.000,1.000,0.000,0.0000,0
5,5,58498,cezarscerbina,thank you for your help all the issues were re...,thank you for your help all the issues were re...,3,4,0.000,0.537,0.463,0.7096,0
6,6,61689,bunq,at bunq privacy is sacred which is why we neve...,at bunq privacy is sacred which is why we neve...,4,1,0.000,0.878,0.122,0.6486,2
7,7,29833,nadiadercourt,youre so efficient thanks,youre so efficient thanks,3,3,0.000,0.242,0.758,0.7408,2
8,8,36524,revolutbusiness,hello there really sorry to hear about your tr...,hello there really sorry to hear about your tr...,4,1,0.190,0.708,0.102,-0.2484,0
9,9,87278,tompickering,go for it support has been good so far althou...,go for it support has been good so far althou...,3,1,0.044,0.856,0.100,0.2382,3


In [415]:
#df_test7.to_csv('test_preds7_xgb.csv')

# Scratch

In [2]:
df = pd.read_csv('custom_out2.csv')
df.dropna(axis=1,inplace=True)

/Users/tawneykirkland/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
## Filter down to non-Revolut tweets (intention is to understand tweets from customers)
df_eng = df[(df['username'] != 'revolutapp') & (df['language'] == 'en')]

In [4]:
def clean_text(text):
    '''Make text lowercase, remove punctuation and remove reference to revolutapp'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('revolutapp', '', text)
    return text

df_eng['tweet'] = df_eng.tweet.map(clean_text)

<ipython-input-4-53ab4f15f761>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_eng['tweet'] = df_eng.tweet.map(clean_text)


In [5]:
## Remove emojis
df_eng['tweet'] = df_eng['tweet'].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))

<ipython-input-5-77c184c069e8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_eng['tweet'] = df_eng['tweet'].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))


In [6]:
with option_context('display.max_colwidth', 600):
    display(df_eng.sample(2))

,id,conversation_id,created_at,date,time,timezone,user_id,username,tweet,language,...,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,link,retweet,video,reply_to
204075,1232381353158696960,1232227847760154625,2020-02-25 14:06:25 EST,2020-02-25,14:06:25,-500,32118861,_hgielyak,i cant even access that as there is no option,en,...,[],1,0,0,[],[],https://twitter.com/_hgielyak/status/1232381353158696960,False,0,"[{'screen_name': 'RevolutApp', 'name': 'Revolut', 'id': '2676313880'}]"
126458,1278779484263546881,1278644412290142208,2020-07-02 15:56:02 EDT,2020-07-02,15:56:02,-500,268786335,jackyw0ng,mastercard how to get one,en,...,[],1,0,0,[],[],https://twitter.com/jackyw0ng/status/1278779484263546881,False,0,"[{'screen_name': 'RevolutApp', 'name': 'Revolut', 'id': '2676313880'}, {'screen_name': 'Mastercard', 'name': 'Mastercard', 'id': '75014376'}]"


In [7]:
## Remove known observations that will not add value
df_eng = df_eng[(df_eng['tweet'] != ' followed') & (df_eng['tweet'] != 'followed ')]

In [8]:
df_short = df_eng.sample(60)

In [9]:
df_short['tweet'] = df_short['tweet'].map(lambda x: x.strip())

In [11]:
# clean_tweet(df_short['tweet'])

In [12]:
df_short

,id,conversation_id,created_at,date,time,timezone,user_id,username,tweet,language,...,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,link,retweet,video,reply_to
10988,1355165887049953280,1355165808087998464,2021-01-29 09:48:19 EST,2021-01-29,09:48:19,-500,912464145278291968,olivertrigg3,riu1986 not allowing buys on gme so good luck...,en,...,[],0,0,0,[],[],https://twitter.com/OliverTrigg3/status/135516...,False,0,"[{'screen_name': 'riu1986', 'name': 'Ieronim',..."
174086,1251028036142522368,1251028036142522368,2020-04-17 02:01:41 EDT,2020-04-17,02:01:41,-500,432873596,golfjimb,hey dudes if im transferring to a foreign acco...,en,...,[],1,0,0,[],[],https://twitter.com/GolfJimB/status/1251028036...,False,0,[]
158881,1262322358553116672,1262322358553116672,2020-05-18 06:01:17 EDT,2020-05-18,06:01:17,-500,243568454,stratorob,here our overview of the top 100 digital banks...,en,...,['https://pbs.twimg.com/media/EYSrfLkXgAE5P1x....,1,2,2,"['fintech', 'ai', 'cloud', 'vr']",[],https://twitter.com/stratorob/status/126232235...,False,1,[]
107531,1290586311053709314,1278644412290142208,2020-08-04 05:52:09 EDT,2020-08-04,05:52:09,-500,1290576449460731904,daveree21737887,origami27 dont you believe it over 24 hours a...,en,...,[],0,0,0,[],[],https://twitter.com/DaveRee21737887/status/129...,False,0,"[{'screen_name': 'RevolutApp', 'name': 'Revolu..."
121321,1281657447866748928,1281657447866748928,2020-07-10 14:32:02 EDT,2020-07-10,14:32:02,-500,920780004170063873,abhy_finn,so yeah dont have access to my funds and bank ...,en,...,[],1,0,0,['revolut'],[],https://twitter.com/abhy_finn/status/128165744...,False,0,[]
58078,1323390979899088905,1323390979899088905,2020-11-02 17:26:11 EST,2020-11-02,17:26:11,-500,229647209,jennrockstar201,my acc is not working 3 days now what is the l...,en,...,[],1,0,0,[],[],https://twitter.com/Jennrockstar201/status/132...,False,0,[]
62430,1321505107037609984,1320713422242283520,2020-10-28 13:32:24 EDT,2020-10-28,13:32:24,-500,1319917098089537537,cicconeraphael,its not enough please respond me in the chat,en,...,[],0,0,0,[],[],https://twitter.com/CicconeRaphael/status/1321...,False,0,"[{'screen_name': 'RevolutApp', 'name': 'Revolu..."
34830,1339164463921041410,1339156240446926849,2020-12-16 06:04:23 EST,2020-12-16,06:04:23,-500,158714419,yemi_s,ukgoldeka you can try barter or,en,...,[],0,0,0,[],[],https://twitter.com/Yemi_S/status/133916446392...,False,0,"[{'screen_name': 'ukgoldeka', 'name': 'E.K.A',..."
156683,1263484767187677185,1263484767187677185,2020-05-21 11:00:17 EDT,2020-05-21,11:00:17,-500,1246026218362847232,alexandradan12,until last day i was highly recommending revol...,en,...,[],0,0,0,[],[],https://twitter.com/AlexandraDan12/status/1263...,False,0,[]
11555,1355085787738624001,1355085787738624001,2021-01-29 04:30:02 EST,2021-01-29,04:30:02,-500,2698110444,techrechayden,have to say that is the worst that is all,en,...,[],0,0,2,[],[],https://twitter.com/TechRecHayden/status/13550...,False,0,[]


In [13]:
tweets = [tweet for tweet in df_short['tweet']]

In [14]:
from textblob import TextBlob

In [15]:
def tweet_sentiment(tweets):
    for tweet in tweets:
        print(tweet,TextBlob(tweet).sentiment)

In [16]:
tweet_sentiment(tweets)

riu1986  not allowing buys on gme so good luck with that Sentiment(polarity=0.7, subjectivity=0.6000000000000001)
hey dudes if im transferring to a foreign account australian does it make a difference if i convert my gbp to aud first is it then like a local transfer cause it said transfer expected weds bit slow im on metal too Sentiment(polarity=-0.04583333333333334, subjectivity=0.20972222222222223)
here our overview of the top 100 digital banks congrats to all  httpstcoihvgtjxgeh  kcbgroup morganstanley  sabbbank monzo bancopichincha bankia scbthailand usbank stanchart   fintech ai cloud vr  team  httpstconubukt0xzv Sentiment(polarity=0.25, subjectivity=0.25)
origami27 dont you believe it  over 24 hours and no help at all  cant access account chat is fkd  this how runs on banks start Sentiment(polarity=0.0, subjectivity=0.0)
so yeah dont have access to my funds and bank statement because revolut app crashed their team can only be reached through app and theyre useless way to go Senti

In [17]:
df_short['textblob'] = df_short['tweet'].map(lambda x: TextBlob(x).sentiment)
df_short['polarity'] = df_short['tweet'].map(lambda x: TextBlob(x).polarity)
df_short['subjectivity'] = df_short['tweet'].map(lambda x: TextBlob(x).subjectivity)

In [18]:
import spacy
import contextualSpellCheck

nlp = spacy.load('en')
checker = contextualSpellCheck.contextualSpellCheck.ContextualSpellCheck()
nlp.add_pipe(checker)

In [19]:
for tweet in df_short['tweet']:
    df2 = pd.DataFrame(nlp(tweet))

In [20]:
from nltk.util import ngrams

from collections import Counter

from textblob import TextBlob

from nltk.corpus import stopwords
stop = stopwords.words('english')

In [21]:
from nltk.tokenize import MWETokenizer

In [22]:
mwe = MWETokenizer([('debit','card'),('business','account'),
                    ('savings','account'), ('bank','account'),
                    ('doge','coin'), ('challenger','bank'),('my','account')])

#df_short['mwe'] = df_short.tweet.apply(mwe.tokenize(word_tokenize(df_short['tweet']))

#mwe_tweet = [mwe.tokenize(word_tokenize(tweets)) for tweet in tweets]

In [23]:
def mwe_tweet(tweet):
    mwe_tok = mwe.tokenize(word_tokenize(tweet))
    
    return mwe_tok

In [24]:
df_short['mwe_token'] = df_short.tweet.map(mwe_tweet) 

In [25]:
df_short

,id,conversation_id,created_at,date,time,timezone,user_id,username,tweet,language,...,hashtags,cashtags,link,retweet,video,reply_to,textblob,polarity,subjectivity,mwe_token
10988,1355165887049953280,1355165808087998464,2021-01-29 09:48:19 EST,2021-01-29,09:48:19,-500,912464145278291968,olivertrigg3,riu1986 not allowing buys on gme so good luck...,en,...,[],[],https://twitter.com/OliverTrigg3/status/135516...,False,0,"[{'screen_name': 'riu1986', 'name': 'Ieronim',...","(0.7, 0.6000000000000001)",0.700000,0.600000,"[riu1986, not, allowing, buys, on, gme, so, go..."
174086,1251028036142522368,1251028036142522368,2020-04-17 02:01:41 EDT,2020-04-17,02:01:41,-500,432873596,golfjimb,hey dudes if im transferring to a foreign acco...,en,...,[],[],https://twitter.com/GolfJimB/status/1251028036...,False,0,[],"(-0.04583333333333334, 0.20972222222222223)",-0.045833,0.209722,"[hey, dudes, if, im, transferring, to, a, fore..."
158881,1262322358553116672,1262322358553116672,2020-05-18 06:01:17 EDT,2020-05-18,06:01:17,-500,243568454,stratorob,here our overview of the top 100 digital banks...,en,...,"['fintech', 'ai', 'cloud', 'vr']",[],https://twitter.com/stratorob/status/126232235...,False,1,[],"(0.25, 0.25)",0.250000,0.250000,"[here, our, overview, of, the, top, 100, digit..."
107531,1290586311053709314,1278644412290142208,2020-08-04 05:52:09 EDT,2020-08-04,05:52:09,-500,1290576449460731904,daveree21737887,origami27 dont you believe it over 24 hours a...,en,...,[],[],https://twitter.com/DaveRee21737887/status/129...,False,0,"[{'screen_name': 'RevolutApp', 'name': 'Revolu...","(0.0, 0.0)",0.000000,0.000000,"[origami27, dont, you, believe, it, over, 24, ..."
121321,1281657447866748928,1281657447866748928,2020-07-10 14:32:02 EDT,2020-07-10,14:32:02,-500,920780004170063873,abhy_finn,so yeah dont have access to my funds and bank ...,en,...,['revolut'],[],https://twitter.com/abhy_finn/status/128165744...,False,0,[],"(-0.25, 0.6)",-0.250000,0.600000,"[so, yeah, dont, have, access, to, my, funds, ..."
58078,1323390979899088905,1323390979899088905,2020-11-02 17:26:11 EST,2020-11-02,17:26:11,-500,229647209,jennrockstar201,my acc is not working 3 days now what is the l...,en,...,[],[],https://twitter.com/Jennrockstar201/status/132...,False,0,[],"(0.5, 0.9)",0.500000,0.900000,"[my, acc, is, not, working, 3, days, now, what..."
62430,1321505107037609984,1320713422242283520,2020-10-28 13:32:24 EDT,2020-10-28,13:32:24,-500,1319917098089537537,cicconeraphael,its not enough please respond me in the chat,en,...,[],[],https://twitter.com/CicconeRaphael/status/1321...,False,0,"[{'screen_name': 'RevolutApp', 'name': 'Revolu...","(0.0, 0.5)",0.000000,0.500000,"[its, not, enough, please, respond, me, in, th..."
34830,1339164463921041410,1339156240446926849,2020-12-16 06:04:23 EST,2020-12-16,06:04:23,-500,158714419,yemi_s,ukgoldeka you can try barter or,en,...,[],[],https://twitter.com/Yemi_S/status/133916446392...,False,0,"[{'screen_name': 'ukgoldeka', 'name': 'E.K.A',...","(0.0, 0.0)",0.000000,0.000000,"[ukgoldeka, you, can, try, barter, or]"
156683,1263484767187677185,1263484767187677185,2020-05-21 11:00:17 EDT,2020-05-21,11:00:17,-500,1246026218362847232,alexandradan12,until last day i was highly recommending revol...,en,...,[],[],https://twitter.com/AlexandraDan12/status/1263...,False,0,[],"(0.032, 0.5713333333333332)",0.032000,0.571333,"[until, last, day, i, was, highly, recommendin..."
11555,1355085787738624001,1355085787738624001,2021-01-29 04:30:02 EST,2021-01-29,04:30:02,-500,2698110444,techrechayden,have to say that is the worst that is all,en,...,[],[],https://twitter.com/TechRecHayden/status/13550...,False,0,[],"(-1.0, 1.0)",-1.000000,1.000000,"[have, to, say, that, is, the, worst, that, is..."


In [27]:
#x_tr['tweet'] = x_tr.tweet.map(clean_text)

In [28]:
import numpy as np
mwe_array = np.asarray(mwe_list)

NameError: name 'mwe_list' is not defined

In [ ]:
mwe_array

In [29]:
mwe_tokens = (lambda x: 
                 x.replace('bank account', 'bank_account')
                 .replace('challenger bank', 'challenger_bank')
                 .replace('my account', 'my_account')
                 .replace('i cant', 'i_cant'))
                 

df_short['tweet'] = df_short.tweet.map(mwe_tokens)

In [30]:
df_short

,id,conversation_id,created_at,date,time,timezone,user_id,username,tweet,language,...,hashtags,cashtags,link,retweet,video,reply_to,textblob,polarity,subjectivity,mwe_token
10988,1355165887049953280,1355165808087998464,2021-01-29 09:48:19 EST,2021-01-29,09:48:19,-500,912464145278291968,olivertrigg3,riu1986 not allowing buys on gme so good luck...,en,...,[],[],https://twitter.com/OliverTrigg3/status/135516...,False,0,"[{'screen_name': 'riu1986', 'name': 'Ieronim',...","(0.7, 0.6000000000000001)",0.700000,0.600000,"[riu1986, not, allowing, buys, on, gme, so, go..."
174086,1251028036142522368,1251028036142522368,2020-04-17 02:01:41 EDT,2020-04-17,02:01:41,-500,432873596,golfjimb,hey dudes if im transferring to a foreign acco...,en,...,[],[],https://twitter.com/GolfJimB/status/1251028036...,False,0,[],"(-0.04583333333333334, 0.20972222222222223)",-0.045833,0.209722,"[hey, dudes, if, im, transferring, to, a, fore..."
158881,1262322358553116672,1262322358553116672,2020-05-18 06:01:17 EDT,2020-05-18,06:01:17,-500,243568454,stratorob,here our overview of the top 100 digital banks...,en,...,"['fintech', 'ai', 'cloud', 'vr']",[],https://twitter.com/stratorob/status/126232235...,False,1,[],"(0.25, 0.25)",0.250000,0.250000,"[here, our, overview, of, the, top, 100, digit..."
107531,1290586311053709314,1278644412290142208,2020-08-04 05:52:09 EDT,2020-08-04,05:52:09,-500,1290576449460731904,daveree21737887,origami27 dont you believe it over 24 hours a...,en,...,[],[],https://twitter.com/DaveRee21737887/status/129...,False,0,"[{'screen_name': 'RevolutApp', 'name': 'Revolu...","(0.0, 0.0)",0.000000,0.000000,"[origami27, dont, you, believe, it, over, 24, ..."
121321,1281657447866748928,1281657447866748928,2020-07-10 14:32:02 EDT,2020-07-10,14:32:02,-500,920780004170063873,abhy_finn,so yeah dont have access to my funds and bank ...,en,...,['revolut'],[],https://twitter.com/abhy_finn/status/128165744...,False,0,[],"(-0.25, 0.6)",-0.250000,0.600000,"[so, yeah, dont, have, access, to, my, funds, ..."
58078,1323390979899088905,1323390979899088905,2020-11-02 17:26:11 EST,2020-11-02,17:26:11,-500,229647209,jennrockstar201,my acc is not working 3 days now what is the l...,en,...,[],[],https://twitter.com/Jennrockstar201/status/132...,False,0,[],"(0.5, 0.9)",0.500000,0.900000,"[my, acc, is, not, working, 3, days, now, what..."
62430,1321505107037609984,1320713422242283520,2020-10-28 13:32:24 EDT,2020-10-28,13:32:24,-500,1319917098089537537,cicconeraphael,its not enough please respond me in the chat,en,...,[],[],https://twitter.com/CicconeRaphael/status/1321...,False,0,"[{'screen_name': 'RevolutApp', 'name': 'Revolu...","(0.0, 0.5)",0.000000,0.500000,"[its, not, enough, please, respond, me, in, th..."
34830,1339164463921041410,1339156240446926849,2020-12-16 06:04:23 EST,2020-12-16,06:04:23,-500,158714419,yemi_s,ukgoldeka you can try barter or,en,...,[],[],https://twitter.com/Yemi_S/status/133916446392...,False,0,"[{'screen_name': 'ukgoldeka', 'name': 'E.K.A',...","(0.0, 0.0)",0.000000,0.000000,"[ukgoldeka, you, can, try, barter, or]"
156683,1263484767187677185,1263484767187677185,2020-05-21 11:00:17 EDT,2020-05-21,11:00:17,-500,1246026218362847232,alexandradan12,until last day i was highly recommending revol...,en,...,[],[],https://twitter.com/AlexandraDan12/status/1263...,False,0,[],"(0.032, 0.5713333333333332)",0.032000,0.571333,"[until, last, day, i, was, highly, recommendin..."
11555,1355085787738624001,1355085787738624001,2021-01-29 04:30:02 EST,2021-01-29,04:30:02,-500,2698110444,techrechayden,have to say that is the worst that is all,en,...,[],[],https://twitter.com/TechRecHayden/status/13550...,False,0,[],"(-1.0, 1.0)",-1.000000,1.000000,"[have, to, say, that, is, the, worst, that, is..."


In [31]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.decomposition import NMF

from nltk.corpus import stopwords
from nltk.tokenize import MWETokenizer

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 

In [32]:
stop = stopwords.words('english')
stop.extend(['ive', 'im', 'itd', 'youre', 'hi', 'hello', 'hey', 'eg', 'l', 'h','w','v','u',
             'please','get','dont','still'])

In [33]:
vectorizer = CountVectorizer(stop_words = stop)
doc_word = vectorizer.fit_transform(df_short.tweet)
doc_word.shape

(60, 400)

In [34]:
nmf_model = NMF(5)
doc_topic = nmf_model.fit_transform(doc_word)
doc_topic.shape

(60, 5)

In [35]:
topic_word = nmf_model.components_
topic_word.shape

(5, 400)

In [36]:
words = vectorizer.get_feature_names()
t = nmf_model.components_.argsort(axis=1)[:,-1:-7:-1]
topic_words = [[words[e] for e in l] for l in t]
topic_words

[['i_cant', 'revolut', 'chat', 'help', 'app', 'thanks'],
 ['money', 'my_account', 'make', 'transfer', 'payment', 'failed'],
 ['account', 'transfer', 'phone', 'revolt', 'apple', 'contact'],
 ['card', 'order', 'revolut', 'time', 'weeks', 'live'],
 ['timeframe', 'chat', 'access', 'app', 'cant', 'ago']]

In [37]:
# # Extend for most common English adverbs

stop.extend(['up','so','out','just','now','how','then','more','also','here',
            'well','only','very','even','back','there','down','still','in',
            'as','to','when','never','really','most','on','why','about','over',
            'again','where','right','off','always','today','all','far','long',
            'away','yet','often','ever','however','almost','later','much',
            'once','least','ago','together','around','already','enough','both',
            'maybe','actually','probably','home','of course','perhaps','little',
            'else','sometimes','finally','less','better','early','especially',
            'either','quite','simply','nearly','soon','certainly','quickly',
            'no','recently','before','usually','thus','exactly','hard',
            'particularly','pretty','forward','ok','okay','clearly','indeed',
            'rather','that','tonight','close','suddenly','best','instead',
            'ahead','fast','alone','eventually','directly'])

# # Extend for most common irregular verbs (except pay,lose,send,buy,spend)

stop.extend(['say','make','go','take','come','see','know','get','got','give',
            'find','think','tell','become','show','leave','feel','put','bring',
            'begin','keep','hold','write','stand','hear','let','mean','set','meet',
            'run','sit','speak','lie','lead','read','grow','fall',
            'build','understand','draw','break','cut','rise','drive','wear',
            'choose'])

# ## Extend for most common irregular verb past tense and past participle

# stop.extend(['said','made','gone','went','took','taken','came','saw','seen','knew',
#              'known','gotten','gave','given','found','thought','became',''
#             'find','think','told','became','showed','shown','left','felt','brought',
#             'began','begun','kept','held','wrote','written','stood','heard','let','meant',
#             'met','ran','sat','spoke','spoken','lay','lain','led','grew','grown','fell',
#              'fallen','built','understood','drew','drawn','broke','broken','rose','risen',
#             'drove','driven','wore','worn'])

# ## Extend for prepositions
stop.extend(['without','among'])

# ## Extend for verbs
# stop.extend(['work','look','looking','seem','seems'])

In [38]:
## Remove one and two word tweets
one_word = df_eng[df_eng['tweet'].str.split().str.len()==1].index 
two_word = df_eng[df_eng['tweet'].str.split().str.len()==2].index 
df_final =  df_eng.drop(one_word, axis = 0)
df_final =  df_eng.drop(two_word, axis = 0)